In [93]:
import os
import cv2
import mrcfile
import numpy as np
import pandas as pd
from PIL import Image

In [81]:
folder = '10576'
mrc_folder = "Protein_data/" + folder + "/micrographs"
true_dir = "Protein_data/" + folder + "/ground_truth/particle_coordinates"
fP_dir = "Protein_data/" + folder + "/ground_truth/false_positives"
output_folder = "Protein_data/" + folder + "/changed"

In [75]:
mrc_folder, true_dir, fP_dir, output_folder

('Protein_data/10532/micrographs',
 'Protein_data/10532/ground_truth/particle_coordinates',
 'Protein_data/10532/ground_truth/false_positives',
 'Protein_data/10532/changed')

In [76]:
# Create the output folders if they don't exist
image_output_folder = os.path.join(output_folder, 'images')
mask_output_folder = os.path.join(output_folder, 'masks')
os.makedirs(image_output_folder, exist_ok=True)
os.makedirs(mask_output_folder, exist_ok=True)


In [77]:
os.path.abspath(os.getcwd())

'/home/pathway/mydisk/Project'

In [78]:
os.getcwd()
os.path.exists('Protein_data/10005/micrographs')

True

In [79]:
img_files = os.listdir(mrc_folder)
# print(img_files)
img_files[0].split('.')[0]

'FoilHole_24138324_Data_24136362_24136364_20200224_094950_Fractions_patch_aligned'

In [97]:
 def making_mask_image(mrc_folder,true_dir,fP_dir,output_folder,img_files,folder):
    fi_len = len(img_files)
    te = 0
    for filename in img_files:
        te = te+1;

        if filename.endswith('.mrc') | filename.endswith('.jpg') :
            print(filename)
            f_name = filename.split('.')[0]
            # path to true particle and false positive particle
            true_path = f_name + '.csv'
            # fp_path = f_name + '_false_positives' + '.csv'
            fp_path = f_name + '.csv'
            true_csv = os.path.join(true_dir,true_path)
            false_csv = os.path.join(fP_dir, fp_path)

            # path of the images and reading the images
            image_path = os.path.join(mrc_folder, filename)

            if filename.endswith('.jpg') :
                image_t = Image.open(image_path)
                grayscale_image = image_t.convert("L")
                image = np.array(grayscale_image)

            else :
                image = mrcfile.read(image_path)
            image = image.T
            image = np.rot90(image)

            image = image.astype(np.float32)
            # Normalize the image data to the range of 0-255
            image = (image - image.min()) / (image.max() - image.min()) * 255.0

            # Convert the image data to uint8
            image = image.astype(np.uint8)

            # Create a blank mask image
            mask = np.zeros_like(image)

            # Update the mask at the particle coordinates
            if os.path.exists(true_csv):
                true_df = pd.read_csv(true_csv)
                for _, row in true_df.iterrows():
                    x, y, diameter = int(row['X-Coordinate']), int(row['Y-Coordinate']), int(row['Diameter'])
                    radius = int(diameter / 2)
                    cv2.circle(mask, (x, y), radius, (255, 255, 255), -1)

            # Update the mask at the particle coordinates
            if os.path.exists(false_csv):
                fp_df = pd.read_csv(false_csv)
                for _, row in fp_df.iterrows():
                    x, y, diameter = int(row['X-Coordinate']), int(row['Y-Coordinate']), int(row['Diameter'])
                    radius = int(diameter / 2)
                    cv2.circle(mask, (x, y), radius, (255, 255, 255), -1)

            # save image
            image_filename = f_name + '_' + folder + '.png'
            image_filepath = os.path.join(image_output_folder, image_filename)
            mask_filepath = os.path.join(mask_output_folder, image_filename)

            print(image_filename,mask_filepath)
            print("###",te,"/" ,fi_len,"###")
            cv2.imwrite(image_filepath, image)
            cv2.imwrite(mask_filepath, mask)

In [100]:
folders = ['10240','10289','10291','10345','10387','10389']
# folders = ['10061','10075','10077','10081','10093','10096','10184']
image_output_folder, mask_output_folder
for folder in folders:
    mrc_folder = "Protein_data/" + folder + "/micrographs"
    true_dir = "Protein_data/" + folder + "/ground_truth/particle_coordinates"
    fP_dir = "Protein_data/" + folder + "/ground_truth/false_positives"
    output_folder = "Protein_data/" + folder + "/changed"
    img_files = os.listdir(mrc_folder)
    
    image_output_folder = os.path.join(output_folder, 'images')
    mask_output_folder = os.path.join(output_folder, 'masks')
    os.makedirs(image_output_folder, exist_ok=True)
    os.makedirs(mask_output_folder, exist_ok=True)
    
    making_mask_image(mrc_folder,true_dir,fP_dir,output_folder,img_files,folder)
    

18feb07c_b1g1_00019gr_00051sq_v02_00002hln_00003enn.jpg
18feb07c_b1g1_00019gr_00051sq_v02_00002hln_00003enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00019gr_00051sq_v02_00002hln_00003enn_10240.png
### 1 / 300 ###
18feb07c_b1g1_00018gr_00047sq_v02_00007hln_00003enn.jpg
18feb07c_b1g1_00018gr_00047sq_v02_00007hln_00003enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00018gr_00047sq_v02_00007hln_00003enn_10240.png
### 2 / 300 ###
18feb07c_b1g1_00020gr_00050sq_v02_00012hln_00003enn.jpg
18feb07c_b1g1_00020gr_00050sq_v02_00012hln_00003enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00020gr_00050sq_v02_00012hln_00003enn_10240.png
### 3 / 300 ###
18feb07c_b1g1_00023gr_00031sq_v02_00007hln_00002enn.jpg
18feb07c_b1g1_00023gr_00031sq_v02_00007hln_00002enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00023gr_00031sq_v02_00007hln_00002enn_10240.png
### 4 / 300 ###
18feb07c_b1g1_00022gr_00030sq_v02_00011hln_00004enn.jpg
18feb07c_b1g1_00022gr_00030s

18feb07c_b1g1_00022gr_00030sq_v02_00006hln_00002enn.jpg
18feb07c_b1g1_00022gr_00030sq_v02_00006hln_00002enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00022gr_00030sq_v02_00006hln_00002enn_10240.png
### 37 / 300 ###
18feb07c_b1g1_00021gr_00055sq_v02_00008hln_00005enn.jpg
18feb07c_b1g1_00021gr_00055sq_v02_00008hln_00005enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00021gr_00055sq_v02_00008hln_00005enn_10240.png
### 38 / 300 ###
18feb07c_b1g1_00022gr_00030sq_v02_00020hln_00004enn.jpg
18feb07c_b1g1_00022gr_00030sq_v02_00020hln_00004enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00022gr_00030sq_v02_00020hln_00004enn_10240.png
### 39 / 300 ###
18feb07c_b1g1_00020gr_00042sq_v02_00008hln_00002enn.jpg
18feb07c_b1g1_00020gr_00042sq_v02_00008hln_00002enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00020gr_00042sq_v02_00008hln_00002enn_10240.png
### 40 / 300 ###
18feb07c_b1g1_00018gr_00046sq_v02_00015hln_00003enn.jpg
18feb07c_b1g1_00018gr_00

18feb07c_b1g1_00021gr_00041sq_v02_00005hln_00004enn.jpg
18feb07c_b1g1_00021gr_00041sq_v02_00005hln_00004enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00021gr_00041sq_v02_00005hln_00004enn_10240.png
### 73 / 300 ###
18feb07c_b1g1_00023gr_00031sq_v02_00005hln_00005enn.jpg
18feb07c_b1g1_00023gr_00031sq_v02_00005hln_00005enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00023gr_00031sq_v02_00005hln_00005enn_10240.png
### 74 / 300 ###
18feb07c_b1g1_00020gr_00043sq_v02_00009hln_00002enn.jpg
18feb07c_b1g1_00020gr_00043sq_v02_00009hln_00002enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00020gr_00043sq_v02_00009hln_00002enn_10240.png
### 75 / 300 ###
18feb07c_b1g1_00021gr_00037sq_v02_00007hln_00003enn.jpg
18feb07c_b1g1_00021gr_00037sq_v02_00007hln_00003enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00021gr_00037sq_v02_00007hln_00003enn_10240.png
### 76 / 300 ###
18feb07c_b1g1_00022gr_00029sq_v02_00016hln_00004enn.jpg
18feb07c_b1g1_00022gr_00

18feb07c_b1g1_00022gr_00032sq_v02_00011hln_00005enn.jpg
18feb07c_b1g1_00022gr_00032sq_v02_00011hln_00005enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00022gr_00032sq_v02_00011hln_00005enn_10240.png
### 109 / 300 ###
18feb07c_b1g1_00020gr_00042sq_v02_00003hln_00002enn.jpg
18feb07c_b1g1_00020gr_00042sq_v02_00003hln_00002enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00020gr_00042sq_v02_00003hln_00002enn_10240.png
### 110 / 300 ###
18feb07c_b1g1_00021gr_00037sq_v02_00005hln_00005enn.jpg
18feb07c_b1g1_00021gr_00037sq_v02_00005hln_00005enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00021gr_00037sq_v02_00005hln_00005enn_10240.png
### 111 / 300 ###
18feb07c_b1g1_00022gr_00027sq_v02_00018hln_00002enn.jpg
18feb07c_b1g1_00022gr_00027sq_v02_00018hln_00002enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00022gr_00027sq_v02_00018hln_00002enn_10240.png
### 112 / 300 ###
18feb07c_b1g1_00021gr_00055sq_v02_00002hln_00003enn.jpg
18feb07c_b1g1_00021g

18feb07c_b1g1_00021gr_00055sq_v02_00006hln_00005enn.jpg
18feb07c_b1g1_00021gr_00055sq_v02_00006hln_00005enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00021gr_00055sq_v02_00006hln_00005enn_10240.png
### 145 / 300 ###
18feb07c_b1g1_00021gr_00038sq_v02_00007hln_00004enn.jpg
18feb07c_b1g1_00021gr_00038sq_v02_00007hln_00004enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00021gr_00038sq_v02_00007hln_00004enn_10240.png
### 146 / 300 ###
18feb07c_b1g1_00021gr_00041sq_v02_00018hln_00002enn.jpg
18feb07c_b1g1_00021gr_00041sq_v02_00018hln_00002enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00021gr_00041sq_v02_00018hln_00002enn_10240.png
### 147 / 300 ###
18feb07c_b1g1_00022gr_00027sq_v02_00011hln_00004enn.jpg
18feb07c_b1g1_00022gr_00027sq_v02_00011hln_00004enn_10240.png Protein_data/10240/changed/masks/18feb07c_b1g1_00022gr_00027sq_v02_00011hln_00004enn_10240.png
### 148 / 300 ###
18feb07c_b1g1_00021gr_00038sq_v02_00008hln_00003enn.jpg
18feb07c_b1g1_00021g

...

18jul30a_00013sq_v01_00008hl_00003ed37-b-DW.jpg
18jul30a_00013sq_v01_00008hl_00003ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00013sq_v01_00008hl_00003ed37-b-DW_10387.png
### 150 / 300 ###
18jul30a_00061sq_v01_00010hl_00005ed37-b-DW.jpg
18jul30a_00061sq_v01_00010hl_00005ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00061sq_v01_00010hl_00005ed37-b-DW_10387.png
### 151 / 300 ###
18jul30a_00051sq_v01_00013hl_00004ed37-b-DW.jpg
18jul30a_00051sq_v01_00013hl_00004ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00051sq_v01_00013hl_00004ed37-b-DW_10387.png
### 152 / 300 ###
18jul30a_00009sq_v01_00003hl_00002ed37-b-DW.jpg
18jul30a_00009sq_v01_00003hl_00002ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00009sq_v01_00003hl_00002ed37-b-DW_10387.png
### 153 / 300 ###
18jul30a_00013sq_v01_00003hl_00002ed37-b-DW.jpg
18jul30a_00013sq_v01_00003hl_00002ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00013sq_v01_00003hl_00002ed3

18jul30a_00060sq_v01_00016hl_00002ed37-b-DW.jpg
18jul30a_00060sq_v01_00016hl_00002ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00060sq_v01_00016hl_00002ed37-b-DW_10387.png
### 190 / 300 ###
18jul30a_00011sq_v01_00037hl_00005ed37-b-DW.jpg
18jul30a_00011sq_v01_00037hl_00005ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00011sq_v01_00037hl_00005ed37-b-DW_10387.png
### 191 / 300 ###
18jul30a_00052sq_v01_00034hl_00004ed37-b-DW.jpg
18jul30a_00052sq_v01_00034hl_00004ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00052sq_v01_00034hl_00004ed37-b-DW_10387.png
### 192 / 300 ###
18jul30a_00011sq_v01_00019hl_00004ed37-b-DW.jpg
18jul30a_00011sq_v01_00019hl_00004ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00011sq_v01_00019hl_00004ed37-b-DW_10387.png
### 193 / 300 ###
18jul30a_00059sq_v01_00024hl_00002ed37-b-DW.jpg
18jul30a_00059sq_v01_00024hl_00002ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00059sq_v01_00024hl_00002ed3

18jul30a_00011sq_v01_00004hl_00004ed37-b-DW.jpg
18jul30a_00011sq_v01_00004hl_00004ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00011sq_v01_00004hl_00004ed37-b-DW_10387.png
### 230 / 300 ###
18jul30a_00060sq_v01_00020hl_00005ed37-b-DW.jpg
18jul30a_00060sq_v01_00020hl_00005ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00060sq_v01_00020hl_00005ed37-b-DW_10387.png
### 231 / 300 ###
18jul30a_00038sq_v01_00003hl_00005ed37-b-DW.jpg
18jul30a_00038sq_v01_00003hl_00005ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00038sq_v01_00003hl_00005ed37-b-DW_10387.png
### 232 / 300 ###
18jul30a_00010sq_v01_00046hl_00003ed37-b-DW.jpg
18jul30a_00010sq_v01_00046hl_00003ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00010sq_v01_00046hl_00003ed37-b-DW_10387.png
### 233 / 300 ###
18jul30a_00005sq_v03_00029hl_00002ed37-b-DW.jpg
18jul30a_00005sq_v03_00029hl_00002ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00005sq_v03_00029hl_00002ed3

18jul30a_00052sq_v01_00017hl_00004ed37-b-DW.jpg
18jul30a_00052sq_v01_00017hl_00004ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00052sq_v01_00017hl_00004ed37-b-DW_10387.png
### 270 / 300 ###
18jul30a_00011sq_v01_00007hl_00003ed37-b-DW.jpg
18jul30a_00011sq_v01_00007hl_00003ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00011sq_v01_00007hl_00003ed37-b-DW_10387.png
### 271 / 300 ###
18jul30a_00012sq_v01_00028hl_00003ed37-b-DW.jpg
18jul30a_00012sq_v01_00028hl_00003ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00012sq_v01_00028hl_00003ed37-b-DW_10387.png
### 272 / 300 ###
18jul30a_00011sq_v01_00025hl_00003ed37-b-DW.jpg
18jul30a_00011sq_v01_00025hl_00003ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00011sq_v01_00025hl_00003ed37-b-DW_10387.png
### 273 / 300 ###
18jul30a_00055sq_v01_00012hl_00005ed37-b-DW.jpg
18jul30a_00055sq_v01_00012hl_00005ed37-b-DW_10387.png Protein_data/10387/changed/masks/18jul30a_00055sq_v01_00012hl_00005ed3

FileNotFoundError: [Errno 2] No such file or directory: 'Protein_data/10389/micrographs'

In [ ]:
#10532